# 🐳 Docker for ML Deployment

**Mục tiêu:** Master Docker để deploy ML models

**Nội dung:**
- Docker fundamentals
- Dockerfile for ML projects
- Docker best practices
- Multi-stage builds
- Docker Compose for multi-service
- FastAPI + Docker deployment
- GPU support

**Level:** Intermediate

**Note:** Notebook này chứa code examples và best practices. Actual Docker commands run in terminal.

---

---

# 1. Docker Fundamentals

## 1.1 Why Docker for ML?

### Problems Docker Solves

❌ **Without Docker:**
- "Works on my machine" syndrome
- Dependency hell (CUDA versions, Python versions)
- Hard to reproduce environments
- Difficult deployment

✅ **With Docker:**
- Consistent environment (dev = prod)
- Package dependencies with code
- Easy to share and deploy
- Isolated environments

## 1.2 Core Concepts

### Image vs Container

**Image** = Blueprint (read-only template)
- Contains: OS, libraries, code, dependencies
- Stored in Docker registry (Docker Hub, ECR, etc.)
- Built from Dockerfile

**Container** = Running instance of image
- Isolated process
- Has its own filesystem, network, processes
- Can be started, stopped, deleted

```
Dockerfile → (build) → Image → (run) → Container
```

### Key Commands

```bash
# Images
docker build -t my-image .          # Build image from Dockerfile
docker images                       # List images
docker rmi my-image                 # Remove image

# Containers
docker run -p 8000:8000 my-image   # Run container
docker ps                           # List running containers
docker ps -a                        # List all containers
docker stop container-id            # Stop container
docker rm container-id              # Remove container

# Execute commands in running container
docker exec -it container-id bash   # Interactive shell

# Logs
docker logs container-id            # View logs
docker logs -f container-id         # Follow logs
```

---

# 2. Dockerfile for ML Projects

## 2.1 Basic Dockerfile

### Example: Simple ML API

In [ ]:
%%writefile Dockerfile.basic
# Basic Dockerfile for ML API

# Base image
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Copy requirements first (for caching)
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Expose port
EXPOSE 8000

# Run application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]

### Dockerfile Instructions

| Instruction | Purpose | Example |
|-------------|---------|----------|
| `FROM` | Base image | `FROM python:3.9-slim` |
| `WORKDIR` | Set working directory | `WORKDIR /app` |
| `COPY` | Copy files from host to container | `COPY . .` |
| `RUN` | Execute command during build | `RUN pip install -r requirements.txt` |
| `ENV` | Set environment variable | `ENV MODEL_PATH=/models/model.pt` |
| `EXPOSE` | Document port (not actually open) | `EXPOSE 8000` |
| `CMD` | Default command when container starts | `CMD ["python", "app.py"]` |
| `ENTRYPOINT` | Command that always runs | `ENTRYPOINT ["python"]` |

## 2.2 Optimized Dockerfile for ML

### Best Practices Applied

In [ ]:
%%writefile Dockerfile.optimized
# Optimized Dockerfile for PyTorch ML API

# Use official PyTorch base (includes CUDA if needed)
FROM pytorch/pytorch:2.0.0-cuda11.7-cudnn8-runtime

# Set environment variables
ENV PYTHONUNBUFFERED=1 \
    PYTHONDONTWRITEBYTECODE=1 \
    PIP_NO_CACHE_DIR=1 \
    PIP_DISABLE_PIP_VERSION_CHECK=1

# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Set working directory
WORKDIR /app

# Copy and install Python dependencies FIRST (for caching)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY ./app /app/app
COPY ./models /app/models

# Create non-root user for security
RUN useradd -m -u 1000 appuser && chown -R appuser:appuser /app
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

# Expose port
EXPOSE 8000

# Run application
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]

### 💡 Optimization Techniques

1. **Layer Caching**
   - Copy `requirements.txt` BEFORE code
   - Dependencies change less frequently
   - Faster rebuilds

2. **Slim Base Images**
   - `python:3.9-slim` vs `python:3.9` (5x smaller)
   - Remove unnecessary packages

3. **No Cache Flags**
   - `pip install --no-cache-dir` (smaller image)
   - `apt-get` cleanup after install

4. **Non-root User**
   - Security best practice
   - Prevents privilege escalation

5. **Health Checks**
   - Monitor container health
   - Auto-restart unhealthy containers

## 2.3 Multi-Stage Builds

### Problem
Including build tools (compilers, dev headers) in final image → Large size

### Solution
Build in one stage, copy artifacts to smaller final stage

In [ ]:
%%writefile Dockerfile.multistage
# Multi-stage build for smaller final image

# ============ Stage 1: Builder ============
FROM python:3.9 AS builder

WORKDIR /build

# Install build dependencies
RUN pip install --upgrade pip
COPY requirements.txt .
RUN pip wheel --no-cache-dir --wheel-dir /build/wheels -r requirements.txt

# ============ Stage 2: Runtime ============
FROM python:3.9-slim

WORKDIR /app

# Copy only wheels from builder stage
COPY --from=builder /build/wheels /wheels
COPY --from=builder /build/requirements.txt .

# Install from wheels (faster, no compilation)
RUN pip install --no-cache-dir --no-index --find-links=/wheels -r requirements.txt \
    && rm -rf /wheels

# Copy application
COPY . .

EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]

# Result: Much smaller final image (no build tools)

### Benefits
- **Smaller image**: 1GB → 300MB (typical)
- **Faster deployment**: Less data to transfer
- **More secure**: Fewer attack surfaces
- **Keep build tools out of production**

---

# 3. FastAPI + Docker Example

## 3.1 Simple ML API with FastAPI

In [ ]:
%%writefile app/main.py
# FastAPI ML inference server

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
import torch.nn as nn
import numpy as np
from typing import List

app = FastAPI(title="ML Model API", version="1.0")

# Simple model (replace with your actual model)
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 2)
    
    def forward(self, x):
        return self.fc(x)

# Load model at startup
model = SimpleModel()
model.eval()

# Request/Response models
class PredictionRequest(BaseModel):
    features: List[float]

class PredictionResponse(BaseModel):
    prediction: int
    probabilities: List[float]

@app.get("/")
def root():
    return {"message": "ML Model API", "status": "running"}

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    try:
        # Validate input
        if len(request.features) != 10:
            raise HTTPException(status_code=400, detail="Expected 10 features")
        
        # Convert to tensor
        x = torch.tensor([request.features], dtype=torch.float32)
        
        # Inference
        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits, dim=-1)
            prediction = torch.argmax(probs, dim=-1).item()
        
        return PredictionResponse(
            prediction=prediction,
            probabilities=probs[0].tolist()
        )
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
%%writefile requirements.txt
fastapi==0.104.1
uvicorn[standard]==0.24.0
torch==2.1.0
numpy==1.24.3
pydantic==2.5.0

In [ ]:
%%writefile .dockerignore
# Don't copy these to Docker image
__pycache__
*.pyc
*.pyo
*.pyd
.Python
*.egg-info
.pytest_cache
.git
.gitignore
*.md
notebooks/
tests/
.venv/
venv/
*.ipynb

## 3.2 Build and Run

### Terminal Commands

```bash
# Build image
docker build -t ml-api:v1 .

# Run container
docker run -d \
  --name ml-api \
  -p 8000:8000 \
  ml-api:v1

# Check logs
docker logs -f ml-api

# Test API
curl http://localhost:8000/health

# Stop container
docker stop ml-api

# Remove container
docker rm ml-api
```

### Python Test Client

In [ ]:
import requests
import numpy as np

# Test health endpoint
response = requests.get("http://localhost:8000/health")
print(f"Health check: {response.json()}")

# Test prediction
features = np.random.randn(10).tolist()
response = requests.post(
    "http://localhost:8000/predict",
    json={"features": features}
)

print(f"\nPrediction: {response.json()}")

---

# 4. Docker Compose for Multi-Service

## 4.1 Why Docker Compose?

### Scenario
ML system with multiple components:
- API server (FastAPI)
- Redis cache
- PostgreSQL database
- Model inference worker

### Problem
Managing multiple `docker run` commands is tedious

### Solution
Docker Compose = Multi-container orchestration

In [ ]:
%%writefile docker-compose.yml
version: '3.8'

services:
  # API Server
  api:
    build:
      context: .
      dockerfile: Dockerfile
    ports:
      - "8000:8000"
    environment:
      - REDIS_HOST=redis
      - POSTGRES_HOST=postgres
      - MODEL_PATH=/models/model.pt
    volumes:
      - ./models:/models
    depends_on:
      - redis
      - postgres
    restart: unless-stopped
    networks:
      - ml-network

  # Redis Cache
  redis:
    image: redis:7-alpine
    ports:
      - "6379:6379"
    volumes:
      - redis-data:/data
    networks:
      - ml-network

  # PostgreSQL Database
  postgres:
    image: postgres:15-alpine
    environment:
      POSTGRES_USER: mluser
      POSTGRES_PASSWORD: mlpassword
      POSTGRES_DB: mldb
    ports:
      - "5432:5432"
    volumes:
      - postgres-data:/var/lib/postgresql/data
    networks:
      - ml-network

  # Model Inference Worker (optional)
  worker:
    build:
      context: .
      dockerfile: Dockerfile.worker
    environment:
      - REDIS_HOST=redis
    volumes:
      - ./models:/models
    depends_on:
      - redis
    deploy:
      replicas: 2  # Scale workers
    networks:
      - ml-network

volumes:
  redis-data:
  postgres-data:

networks:
  ml-network:
    driver: bridge

## 4.2 Docker Compose Commands

```bash
# Start all services
docker-compose up -d

# View logs
docker-compose logs -f api

# Scale service
docker-compose up -d --scale worker=3

# Stop all services
docker-compose down

# Stop and remove volumes (clean slate)
docker-compose down -v

# Rebuild images
docker-compose build

# View running services
docker-compose ps
```

## 4.3 Key Features

### Environment Variables
```yaml
environment:
  - REDIS_HOST=redis
  - DEBUG=false
```

### Volumes (Persistent Data)
```yaml
volumes:
  - ./models:/models          # Bind mount (host -> container)
  - postgres-data:/var/lib/postgresql/data  # Named volume
```

### Networks (Service Communication)
```yaml
networks:
  - ml-network
```
Services can reach each other by service name (e.g., `http://api:8000`)

### Dependencies
```yaml
depends_on:
  - redis
  - postgres
```
Start order: redis, postgres → api

### Health Checks
```yaml
healthcheck:
  test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
  interval: 30s
  timeout: 3s
  retries: 3
```

---

# 5. GPU Support with Docker

## 5.1 NVIDIA Docker

### Requirements
1. NVIDIA GPU
2. NVIDIA drivers installed
3. `nvidia-docker2` package

### Installation
```bash
# Install nvidia-docker
distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
curl -s -L https://nvidia.github.io/nvidia-docker/gpgkey | sudo apt-key add -
curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.list | \
  sudo tee /etc/apt/sources.list.d/nvidia-docker.list

sudo apt-get update
sudo apt-get install -y nvidia-docker2
sudo systemctl restart docker
```

### Dockerfile with CUDA

In [ ]:
%%writefile Dockerfile.gpu
# GPU-enabled Dockerfile

FROM nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04

# Install Python
RUN apt-get update && apt-get install -y \
    python3.10 \
    python3-pip \
    && rm -rf /var/lib/apt/lists/*

WORKDIR /app

# Install PyTorch with CUDA support
COPY requirements.txt .
RUN pip install --no-cache-dir torch torchvision --index-url https://download.pytorch.org/whl/cu118
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 8000
CMD ["python3", "app.py"]

### Run with GPU

```bash
# Run with GPU access
docker run --gpus all -p 8000:8000 ml-api-gpu:v1

# Specify number of GPUs
docker run --gpus 2 -p 8000:8000 ml-api-gpu:v1

# Specify GPU device IDs
docker run --gpus '"device=0,1"' -p 8000:8000 ml-api-gpu:v1
```

### Docker Compose with GPU

```yaml
version: '3.8'

services:
  ml-api:
    build: .
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu]
```

---

# 6. Best Practices

## 6.1 Image Size Optimization

### Tips

1. **Use slim/alpine base images**
   ```dockerfile
   FROM python:3.9-slim  # ~150MB vs python:3.9 ~900MB
   ```

2. **Multi-stage builds**
   - Build stage: Install dependencies
   - Runtime stage: Copy only artifacts

3. **Minimize layers**
   ```dockerfile
   # ❌ Multiple layers
   RUN apt-get update
   RUN apt-get install -y package1
   RUN apt-get install -y package2
   
   # ✅ Single layer
   RUN apt-get update && apt-get install -y \
       package1 \
       package2 \
       && rm -rf /var/lib/apt/lists/*
   ```

4. **Clean up**
   ```dockerfile
   RUN pip install --no-cache-dir -r requirements.txt
   RUN apt-get clean && rm -rf /var/lib/apt/lists/*
   ```

5. **`.dockerignore`**
   - Exclude unnecessary files
   - Faster builds, smaller context

## 6.2 Security

### Tips

1. **Non-root user**
   ```dockerfile
   RUN useradd -m -u 1000 appuser
   USER appuser
   ```

2. **Scan for vulnerabilities**
   ```bash
   docker scan my-image:latest
   ```

3. **Use official images**
   - Trusted sources
   - Regularly updated

4. **Don't store secrets in image**
   ```bash
   # ✅ Use environment variables
   docker run -e API_KEY=$API_KEY my-image
   
   # ✅ Or Docker secrets
   docker secret create api_key api_key.txt
   ```

## 6.3 Development Workflow

### Hot Reload with Volumes

```bash
# Mount code directory
docker run -v $(pwd)/app:/app/app -p 8000:8000 ml-api:v1
```

Changes in host `app/` folder reflect in container immediately!

### Docker Compose Dev Setup

```yaml
services:
  api:
    build: .
    volumes:
      - ./app:/app/app  # Hot reload
    environment:
      - DEBUG=true
    command: uvicorn app.main:app --reload --host 0.0.0.0
```

---

# 🎯 Key Takeaways

## Essential Commands

```bash
# Build
docker build -t my-image:v1 .

# Run
docker run -d -p 8000:8000 --name my-container my-image:v1

# Logs
docker logs -f my-container

# Execute
docker exec -it my-container bash

# Stop & Remove
docker stop my-container
docker rm my-container

# Docker Compose
docker-compose up -d      # Start
docker-compose down       # Stop
docker-compose logs -f    # Logs
```

## Dockerfile Best Practices

1. ✅ **Order matters**: Put frequently changing files last
2. ✅ **Use `.dockerignore`**: Exclude unnecessary files
3. ✅ **Combine RUN commands**: Minimize layers
4. ✅ **Clean up**: Remove caches and temp files
5. ✅ **Multi-stage builds**: Smaller final images
6. ✅ **Non-root user**: Security
7. ✅ **Health checks**: Monitor container health

## Common Patterns

### Basic ML API
```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0"]
```

### With GPU Support
```dockerfile
FROM nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04
# ... install PyTorch with CUDA
```

```bash
docker run --gpus all my-gpu-image
```

### Multi-Service with Compose
```yaml
services:
  api:
    build: .
    ports: ["8000:8000"]
  redis:
    image: redis:alpine
  postgres:
    image: postgres:15
```

## Troubleshooting

```bash
# Check logs
docker logs container-name

# Inspect container
docker inspect container-name

# Interactive shell
docker exec -it container-name bash

# Check resource usage
docker stats

# Clean up
docker system prune -a  # Remove unused containers, images, networks
```

---

**Next:** timm (PyTorch Image Models)